First, packages are imported.

In [3]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import prim
from ema_workbench import Samplers

Load results from Uncertainty Analysis

In [70]:
uncertainty_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_provinces.pkl')
experiments_and_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_experiments_results.pkl')
aggregated_outcomes = pd.read_pickle(r'../generated_datasets/policy_uncertainty_aggregated.pkl')
uncertainty_results_run = pd.read_pickle(r'../generated_datasets/policy_uncertainty_test.pkl')

# Calculate Robustness Metrics
MOET NOG

Once the evaluation data is built through computational experimentation in step 3, that information is used to calculate policy robustness and to discover vulnerabilities in the existing policy op-tionsbasedontheidentifiedrobustnessmeasure(possiblerobustnessmetricswerediscussedinSec-tion 3.2, but RDM traditionally favors a satisficing measure (Matrosov, Woods, & Harou,2013)). Theprocess of searching for vulnerabilities is known as Scenario Discovery. PRIM (the Patient Rule In-duction Method) is typically identified as the most suitable scenario discovery method for RDM, as itproduces simple rules that accurately determine the factors (both uncertainties and policy settings)which contribute to poor performance (Lempert et al.,2006). This information is used to refine boththemodelandpolicyoptionsinaniterativeprocessuntiltheanalystandotherdecisionmakersreacha policy option or set of options that they can implement to address the problem.

In [71]:
experiments, outcomes = uncertainty_results_run

results_df = pd.DataFrame()
for k in outcomes:
    test_list = []
    for i in range(len(experiments)):
        t = outcomes[k][i].sum()
        test_list.append(t)
    results_df[k]=test_list

results_df

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.2_Expected Number of Deaths,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.3_Expected Number of Deaths,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.4_Expected Number of Deaths,A.5_Expected Annual Damage,A.5_Dike Investment Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0.000000e+00,1.422000e+08,0.000000,0.0,2.343025e+08,0.0,2.598958e+08,4.962273e+07,0.051265,0.000000e+00,1.106877e+07,0.000000,0.000000e+00,7.258061e+07,0.000000,121200000.0,7440.699768
1,0.000000e+00,1.422000e+08,0.000000,0.0,2.343025e+08,0.0,2.007740e+07,4.962273e+07,0.004117,0.000000e+00,1.106877e+07,0.000000,0.000000e+00,7.258061e+07,0.000000,121200000.0,529.613015
2,0.000000e+00,1.422000e+08,0.000000,0.0,2.343025e+08,0.0,2.169477e+07,4.962273e+07,0.008693,0.000000e+00,1.106877e+07,0.000000,0.000000e+00,7.258061e+07,0.000000,121200000.0,1131.697699
3,0.000000e+00,1.422000e+08,0.000000,0.0,2.343025e+08,0.0,1.138822e+08,4.962273e+07,0.028815,0.000000e+00,1.106877e+07,0.000000,0.000000e+00,7.258061e+07,0.000000,121200000.0,3997.055938
4,0.000000e+00,1.422000e+08,0.000000,0.0,2.343025e+08,0.0,2.680297e+08,4.962273e+07,0.083679,0.000000e+00,1.106877e+07,0.000000,0.000000e+00,7.258061e+07,0.000000,121200000.0,12299.407927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66995,0.000000e+00,1.352742e+08,0.000000,0.0,2.160669e+08,0.0,2.618213e+07,4.871687e+07,0.007885,1.605862e+06,7.332080e+06,0.000131,4.768190e+07,7.908444e+07,0.006883,0.0,5617.388654
66996,0.000000e+00,1.352742e+08,0.000000,0.0,2.160669e+08,0.0,2.509358e+07,4.871687e+07,0.007961,7.452276e+06,7.332080e+06,0.000680,0.000000e+00,7.908444e+07,0.000000,0.0,2236.872345
66997,4.610411e+07,1.352742e+08,0.005638,0.0,2.160669e+08,0.0,0.000000e+00,4.871687e+07,0.000000,1.455608e+06,7.332080e+06,0.000139,0.000000e+00,7.908444e+07,0.000000,0.0,1140.688872
66998,0.000000e+00,1.352742e+08,0.000000,0.0,2.160669e+08,0.0,4.829867e+06,4.871687e+07,0.001320,3.099803e+06,7.332080e+06,0.000227,0.000000e+00,7.908444e+07,0.000000,0.0,438.029693


In [72]:
death_cost_per_person = 6_300_000

# calculate the cost of death so every column has the same unit (€)
results_df["A.1_Deaths_Cost"]=results_df["A.1_Expected Number of Deaths"]*death_cost_per_person
results_df["A.2_Deaths_Cost"]=results_df["A.2_Expected Number of Deaths"]*death_cost_per_person
results_df["A.3_Deaths_Cost"]=results_df["A.3_Expected Number of Deaths"]*death_cost_per_person
results_df["A.4_Deaths_Cost"]=results_df["A.4_Expected Number of Deaths"]*death_cost_per_person
results_df["A.5_Deaths_Cost"]=results_df["A.5_Expected Number of Deaths"]*death_cost_per_person

to_drop = ["A.1_Expected Number of Deaths","A.2_Expected Number of Deaths","A.3_Expected Number of Deaths","A.4_Expected Number of Deaths","A.5_Expected Number of Deaths"]

results_df.drop(to_drop, axis=1, inplace=True)
# results_df['total_cost[€]']= results_df.sum(axis=1)

results_df

,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.2_Expected Annual Damage,A.2_Dike Investment Costs,A.3_Expected Annual Damage,A.3_Dike Investment Costs,A.4_Expected Annual Damage,A.4_Dike Investment Costs,A.5_Expected Annual Damage,A.5_Dike Investment Costs,RfR Total Costs,Expected Evacuation Costs,A.1_Deaths_Cost,A.2_Deaths_Cost,A.3_Deaths_Cost,A.4_Deaths_Cost,A.5_Deaths_Cost
0,0.000000e+00,1.422000e+08,0.0,2.343025e+08,2.598958e+08,4.962273e+07,0.000000e+00,1.106877e+07,0.000000e+00,7.258061e+07,121200000.0,7440.699768,0.000000,0.0,322971.446503,0.000000,0.000000
1,0.000000e+00,1.422000e+08,0.0,2.343025e+08,2.007740e+07,4.962273e+07,0.000000e+00,1.106877e+07,0.000000e+00,7.258061e+07,121200000.0,529.613015,0.000000,0.0,25938.556696,0.000000,0.000000
2,0.000000e+00,1.422000e+08,0.0,2.343025e+08,2.169477e+07,4.962273e+07,0.000000e+00,1.106877e+07,0.000000e+00,7.258061e+07,121200000.0,1131.697699,0.000000,0.0,54763.469958,0.000000,0.000000
3,0.000000e+00,1.422000e+08,0.0,2.343025e+08,1.138822e+08,4.962273e+07,0.000000e+00,1.106877e+07,0.000000e+00,7.258061e+07,121200000.0,3997.055938,0.000000,0.0,181533.542707,0.000000,0.000000
4,0.000000e+00,1.422000e+08,0.0,2.343025e+08,2.680297e+08,4.962273e+07,0.000000e+00,1.106877e+07,0.000000e+00,7.258061e+07,121200000.0,12299.407927,0.000000,0.0,527180.556318,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66995,0.000000e+00,1.352742e+08,0.0,2.160669e+08,2.618213e+07,4.871687e+07,1.605862e+06,7.332080e+06,4.768190e+07,7.908444e+07,0.0,5617.388654,0.000000,0.0,49677.442362,827.562598,43359.905646
66996,0.000000e+00,1.352742e+08,0.0,2.160669e+08,2.509358e+07,4.871687e+07,7.452276e+06,7.332080e+06,0.000000e+00,7.908444e+07,0.0,2236.872345,0.000000,0.0,50151.312206,4283.788780,0.000000
66997,4.610411e+07,1.352742e+08,0.0,2.160669e+08,0.000000e+00,4.871687e+07,1.455608e+06,7.332080e+06,0.000000e+00,7.908444e+07,0.0,1140.688872,35518.629332,0.0,0.000000,874.937299,0.000000
66998,0.000000e+00,1.352742e+08,0.0,2.160669e+08,4.829867e+06,4.871687e+07,3.099803e+06,7.332080e+06,0.000000e+00,7.908444e+07,0.0,438.029693,0.000000,0.0,8316.725584,1432.709180,0.000000


### Gelderland - Overijssel Split

In [73]:
def combine_columns_province(dataframe, name):
    combined_columns_gelderland = []
    combined_columns_overijssel = []
    for x in dataframe.columns:
        if name in x:
            if x.startswith('A.1') or x.startswith('A.2') or x.startswith('A.3'):
                combined_columns_gelderland.append(x)
            if x.startswith('A.4') or x.startswith('A.5'):
                combined_columns_overijssel.append(x)


    dataframe['Total ' + str(name) + ' Gelderland'] = dataframe[combined_columns_gelderland].sum(axis=1)
    dataframe['Total ' + str(name) + ' Overijssel'] = dataframe[combined_columns_overijssel].sum(axis=1)


In [74]:
provinces = results_df.copy()

combine_columns_province(provinces, 'Expected Annual Damage')
combine_columns_province(provinces, 'Dike Investment Costs')
combine_columns_province(provinces, 'Deaths_Cost')
#combine_columns_province(provinces, 'RfR Total Costs')
#combine_columns_province(provinces, 'Expected Evacuation Costs')

In [93]:
aggregated_outcomes_provinces = pd.concat([experiments[['scenario','policy']],provinces], axis=1).iloc[:,2:]

for x in aggregated_outcomes_provinces.columns:

        if x.startswith('A.1') or x.startswith('A.2') or x.startswith('A.3') or x.startswith('A.4') or x.startswith('A.5'):
            aggregated_outcomes_provinces.drop(x, axis=1, inplace=True)
        else:
            pass

aggregated_outcomes_provinces['total_cost[€]']= aggregated_outcomes_provinces.sum(axis=1)
overijssel_sum = ['RfR Total Costs','Expected Evacuation Costs','Total Expected Annual Damage Overijssel','Total Dike Investment Costs Overijssel','Total Deaths_Cost Overijssel']
aggregated_outcomes_provinces['total_cost[€]_overijssel']= aggregated_outcomes_provinces.loc[:,overijssel_sum].sum(axis=1)

#with open(r'../generated_datasets/policy_uncertainty_provinces.pkl', 'wb') as file_pi:
 #   pickle.dump(aggregated_outcomes_provinces, file_pi)
aggregated_outcomes_provinces

,RfR Total Costs,Expected Evacuation Costs,Total Expected Annual Damage Gelderland,Total Expected Annual Damage Overijssel,Total Dike Investment Costs Gelderland,Total Dike Investment Costs Overijssel,Total Deaths_Cost Gelderland,Total Deaths_Cost Overijssel,total_cost[€],total_cost[€]_overijssel
0,121200000.0,7440.699768,2.598958e+08,0.000000e+00,4.261252e+08,8.364938e+07,322971.446503,0.000000,8.912008e+08,2.048568e+08
1,121200000.0,529.613015,2.007740e+07,0.000000e+00,4.261252e+08,8.364938e+07,25938.556696,0.000000,6.510785e+08,2.048499e+08
2,121200000.0,1131.697699,2.169477e+07,0.000000e+00,4.261252e+08,8.364938e+07,54763.469958,0.000000,6.527253e+08,2.048505e+08
3,121200000.0,3997.055938,1.138822e+08,0.000000e+00,4.261252e+08,8.364938e+07,181533.542707,0.000000,7.450423e+08,2.048534e+08
4,121200000.0,12299.407927,2.680297e+08,0.000000e+00,4.261252e+08,8.364938e+07,527180.556318,0.000000,8.995438e+08,2.048617e+08
...,...,...,...,...,...,...,...,...,...,...
66995,0.0,5617.388654,2.618213e+07,4.928776e+07,4.000580e+08,8.641652e+07,49677.442362,44187.468244,5.620439e+08,1.357541e+08
66996,0.0,2236.872345,2.509358e+07,7.452276e+06,4.000580e+08,8.641652e+07,50151.312206,4283.788780,5.190771e+08,9.387531e+07
66997,0.0,1140.688872,4.610411e+07,1.455608e+06,4.000580e+08,8.641652e+07,35518.629332,874.937299,5.340718e+08,8.787414e+07
66998,0.0,438.029693,4.829867e+06,3.099803e+06,4.000580e+08,8.641652e+07,8316.725584,1432.709180,4.944144e+08,8.951819e+07


### Generate outcome optimal for Overijssel

In [96]:
n_scenarios_of_interest = 5

results_of_interest = aggregated_outcomes_provinces.sort_values(by='total_cost[€]_overijssel', ascending=True).head(n_scenarios_of_interest)

with open(r'..\generated_datasets\final_policies_overijssel.pkl','wb') as pickle_file:
    pickle.dump(results_of_interest, pickle_file)

results_of_interest

,RfR Total Costs,Expected Evacuation Costs,Total Expected Annual Damage Gelderland,Total Expected Annual Damage Overijssel,Total Dike Investment Costs Gelderland,Total Dike Investment Costs Overijssel,Total Deaths_Cost Gelderland,Total Deaths_Cost Overijssel,total_cost[€],total_cost[€]_overijssel
10153,0.0,0.0,1.466074e+09,0.0,2.399554e+08,0.0,1.808713e+07,0.0,1.724117e+09,0.0
10004,0.0,0.0,7.806502e+08,0.0,2.399554e+08,0.0,9.854233e+06,0.0,1.030460e+09,0.0
10718,0.0,0.0,1.774258e+09,0.0,2.399554e+08,0.0,1.769794e+07,0.0,2.031912e+09,0.0
10717,0.0,0.0,1.229246e+09,0.0,2.399554e+08,0.0,1.794797e+07,0.0,1.487149e+09,0.0
10942,0.0,0.0,2.527842e+09,0.0,2.399554e+08,0.0,1.251653e+07,0.0,2.780314e+09,0.0


### Generate outcome optimal for Gelderland and Overijssel combined.

In [97]:
n_scenarios_of_interest = 5

results_of_interest = aggregated_outcomes_provinces.sort_values(by='total_cost[€]', ascending=True).head(n_scenarios_of_interest)

with open(r'..\generated_datasets\final_policies.pkl','wb') as pickle_file:
    pickle.dump(results_of_interest, pickle_file)

results_of_interest

,RfR Total Costs,Expected Evacuation Costs,Total Expected Annual Damage Gelderland,Total Expected Annual Damage Overijssel,Total Dike Investment Costs Gelderland,Total Dike Investment Costs Overijssel,Total Deaths_Cost Gelderland,Total Deaths_Cost Overijssel,total_cost[€],total_cost[€]_overijssel
46168,0.0,1091.291595,5.996380e+06,1.259604e+07,2.192581e+08,0.0,13130.164696,7226.065362,2.378719e+08,1.260436e+07
46279,0.0,1031.516199,1.359283e+07,1.500558e+07,2.192581e+08,0.0,9379.099391,6215.012226,2.478731e+08,1.501282e+07
46099,0.0,1547.899340,1.040607e+07,1.963150e+07,2.192581e+08,0.0,11660.359848,9365.151806,2.493182e+08,1.964241e+07
46162,0.0,1514.935674,1.686334e+07,1.922814e+07,2.192581e+08,0.0,11557.108157,15437.227596,2.553780e+08,1.924509e+07
46468,0.0,2204.888753,1.354374e+07,2.326346e+07,2.192581e+08,0.0,15023.981315,12454.073843,2.560949e+08,2.327812e+07


### PRIM

In [8]:
x = experiments_and_results.iloc[:, :19]

for var in aggregated_outcomes.iloc[:, 2:].columns:
    var_path = str('../visualizations/Scenario_Discovery/policy_' + var + '_PRIM.png')
    var2_path = str('../visualizations/Scenario_Discovery/policy_' + var + '_PRIM_inspect.png')

    y_total_damage = aggregated_outcomes[var].values
    y_total_damage = y_total_damage > np.percentile(y_total_damage, 75)

    prim_alg_DAM = prim.Prim(x,
                             y_total_damage,
                             threshold=0.70,
                             peel_alpha=0.05, )  # deze variabelen nog tweaken
    box_DAM = prim_alg_DAM.find_box()

    img = box_DAM.show_tradeoff()
    plt.title(var)
    fig = img.get_figure()
    fig.savefig(var_path, bbox_inches='tight')
    fig.clf()

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>